# Weather data

In this assignment, we will read weather data. The data will be processed and the radiance will be calculated using properties of our model. The weather data will be formatted into usable input and stored in a csv file for use in the next assignment.

The script starts by importing the essential Python libraries.

In [2]:
import numpy as np                 
import pandas as pd                   
import matplotlib.pyplot as plt       
import dm4bem                         

Flags are defined to control the behavior of the simulation: controller = True enables a simulated temperature controller. Neglect_air_glass_capacity = True assumes air and glass have no thermal capacity (explained in the previous assignments). Imposed_time_step decides if we force a fixed time step (Δt = 1800 s = 30 min).

In [3]:
controller = True                            
neglect_air_glass_capacity = True            
imposed_time_step = False                    
Δt = 1800                                    

The thermal circuit (a representation of the building’s thermal behavior) is loaded from a CSV file using dm4bem.file2TC. This is the model made in assignment 1. If the controller is activated, two conductances q6 and q14 are set to a large value (1000 W/K) to simulate almost perfect indoor temperature control.

In [4]:
TC = dm4bem.file2TC('./MODEL/TC.csv', name='', auto_number=False)
if controller:
    TC['G']['q6'] = 1e3        
    TC['G']['q14'] = 1e3 

The thermal circuit is converted into a state-space model using dm4bem.tc2ss(). This allows us to simulate the thermal dynamics of the system with matrices.

In [6]:
[As, Bs, Cs, Ds, us] = dm4bem.tc2ss(TC)

 Compute eigenvalues, max stable Δt, and choose time step



In [7]:
λ = np.linalg.eig(As)[0]
Δtmax = 2 * min(-1 / λ)
dm4bem.print_rounded_time('Δtmax', Δtmax)

imposed_time_step = True
dt = Δt if imposed_time_step else dm4bem.round_time(Δtmax)

if dt < 10:
    raise ValueError("Time step is too small. Stopping the script.")

dm4bem.print_rounded_time('dt', dt)

Δtmax = 8413 s = 2.3 h
dt = 1800 s = 30.0 min


Calculate settling time and total duration for simulation

In [8]:
t_settle = 4 * max(-1 / λ)
dm4bem.print_rounded_time('t_settle', t_settle)

duration = np.ceil(t_settle / 3600) * 3600
dm4bem.print_rounded_time('duration', duration)

t_settle = 175031 s = 48.6 h
duration = 176400 s = 49.0 h


Set start and end dates for simulation (May 1 to May 7, 2025). This was our own choice.

In [9]:
start_date = '05-01 12:00:00'
end_date = '05-07 12:00:00'
start_date = '2025-' + start_date
end_date = '2025-' + end_date
print(f'{start_date} \tstart date')
print(f'{end_date} \tend date')

2025-05-01 12:00:00 	start date
2025-05-07 12:00:00 	end date


 Load EPW file, get air temp and radiation, filter for the chosen period. As location we chose the airport of Grenoble. 

In [10]:
filename = './weather_data/FRA_AR_Grenoble_Airport.epw'
[data, meta] = dm4bem.read_epw(filename, coerce_year=None)

weather = data[["temp_air", "dir_n_rad", "dif_h_rad"]]
del data
weather.index = weather.index.map(lambda t: t.replace(year=2025))
weather = weather.loc[start_date:end_date]


 Get outside air temperature as input

In [11]:
To = weather['temp_air']

Define τ and load wall data. The outdoor walls are described in walls_out.csv. The outdoor walls absorb a certain amount of the radiation from the sun. The indoor wall is also import because of the fact that we have a window which lets radiation through. 

In [13]:
τ_gSW = 0.30  # short wave transmitance: reflective blue glass

wall_out = pd.read_csv('./BLDG/walls_out.csv')
w1 = wall_out[wall_out['ID'] == 'w0'] #concrete
w2 = wall_out[wall_out['ID'] == 'w1'] #insulation
wall_in = pd.read_csv('./BLDG/walls_in.csv')
w3 = wall_in[wall_in['ID'] == 'w0'] #concrete

Compute tilted surface irradiance Etot. The orientation of all the walls is assumed to be the same. This is a simplification of the reality to make programming easier.

In [14]:
surface_orientation = {'slope': w1['β'].values[0],
                       'azimuth': w1['γ'].values[0],
                       'latitude': 45}

rad_surf = dm4bem.sol_rad_tilt_surf(weather, surface_orientation, w1['albedo'].values[0])
Etot = rad_surf.sum(axis=1)

Calculate solar flux absorbed by the outdoor walls Φo1, Φo2 and the solar flux of the indoor walls (because of the window) Φi1 (for the concrete + insulation walls of room 1) and Φi2 (for the middle wall). The shortwave radiation properties of walls and glass were defined in the corresponding csv files. α1 is the outdoor absorptivity of a wall material and α0 is the indoor absorptivity of a wall material.

In [15]:
Φo1 = w1['α1'].values[0] * w1['Area'].values[0] * Etot
Φo2 = w2['α1'].values[0] * w2['Area'].values[0] * Etot

S_glass = 12  # m²
Φi1 = τ_gSW * w2['α0'].values[0] * S_glass * Etot
Φi2 = τ_gSW * w3['α0'].values[0] * S_glass * Etot

Set indoor temps: 20°C during day, 16°C at night

In [16]:
Ti_day, Ti_night = 20, 16

Ti1 = pd.Series([Ti_day if 6 <= hour <= 22 else Ti_night for hour in To.index.hour], index=To.index)
Ti2 = pd.Series([Ti_day if 6 <= hour <= 22 else Ti_night for hour in To.index.hour], index=To.index)

 Combine all data in one DataFrame and save into a csv file

In [17]:
input_data_set = pd.DataFrame({'To': To, 'Ti1': Ti1, 'Ti2': Ti2,
                               'Φo1': Φo1, 'Φi1': Φi1, 'Φi2': Φi2,
                               'Φo2': Φo2, 'Etot': Etot})

input_data_set.to_csv('./MODEL/input_data_set.csv')